# **Describe Data:**

https://www.kaggle.com/c/petfinder-adoption-prediction/data

**Problem:**

predict the speed at which a pet is adopted, based on the pet’s listing on PetFinder. Sometimes a profile represents a group of pets. In this case, the speed of adoption is determined by the speed at which all of the pets are adopted. 

**The data included text, tabular, and image data.**

## **Data Fields:**

**PetID** - Unique hash ID of pet profile

**AdoptionSpeed** - Categorical speed of adoption. Lower is faster. **This is the 
value to predict.**

**Type** - Type of animal (1 = Dog, 2 = Cat)

**Name** - Name of pet (Empty if not named)

**Age** - Age of pet when listed, in months

**Breed1** - Primary breed of pet (Refer to BreedLabels dictionary)

**Breed2** - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels 
dictionary)

**Gender** - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents 
group of pets)

**Color1** - Color 1 of pet (Refer to ColorLabels dictionary)

**Color2** - Color 2 of pet (Refer to ColorLabels dictionary)

**Color3** - Color 3 of pet (Refer to ColorLabels dictionary)

**MaturitySize** - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra 

Large, 0 = Not Specified)

**FurLength** - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)

**Vaccinated** - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)

**Dewormed** - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)

**Sterilized** - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)

**Health** - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 
= Not Specified)

**Quantity** - Number of pets represented in profile

**Fee** - Adoption fee (0 = Free)

**State** - State location in Malaysia (Refer to StateLabels dictionary)

**RescuerID** - Unique hash ID of rescuer

**VideoAmt** - Total uploaded videos for this pet

**PhotoAmt** - Total uploaded photos for this pet

**Description** - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.

## **AdoptionSpeed**

The value is determined by how quickly, if at all, a pet is adopted. The values are determined in the 
following way:

**0** - Pet was adopted on the same day as it was listed.

**1** - Pet was adopted between 1 and 7 days (1st week) after being listed.

**2** - Pet was adopted between 8 and 30 days (1st month) after being listed.

**3** - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.

**4** - No adoption after 100 days of being listed. (There are no pets in this 
dataset that waited between 90 and 100 days).






Evaluation criteria:
- Use of tabular data = 50% of the mark 
- Use of text data (Pet Description) or Image Data = 75% of the mark 

- Use of ALL sources = Tabular+Image+Text+Sentiment = 100% of the mark 


Bonus:
- Submission to Kaggle on official Leaderboard = +5
- Top 15% score = +10
- Top 5% score = +20
- Top 3% score = +30

# **Get Data:**

In [ ]:
#!pip install catboost

In [ ]:
import os

In [ ]:
# Install Kaggle API
!pip install --quiet kaggle


if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle

# !mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_json/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

############################################
!sudo apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
#!pip uninstall -y kaggle


In [ ]:
#!pip install --upgrade pip


In [ ]:
#!pip install kaggle==1.5.6

In [ ]:
#!pip install --upgrade kaggle

In [ ]:
#! kaggle competitions download -c petfinder-adoption-prediction

# **Import Needed Library**

In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K

# **Load Tabular Data**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/p2/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/p2/test.csv')
img_size = 256
batch_size = 16

Un Zip Images Zip Folders in a new folder

In [ ]:
#!unzip -uq "/content/drive/MyDrive/Datasets/p2/train_images.zip" -d "/content/drive/MyDrive/Datasets/p2/train_images"

In [ ]:
#!unzip -uq "/content/drive/MyDrive/Datasets/p2/test_images.zip" -d "/content/drive/MyDrive/Datasets/p2/test_images"

#**Feature Extraction Using DenseNet121**

In [ ]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

In [ ]:
def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [ ]:
inp = Input((img_size, img_size, 3))
backbone = DenseNet121(input_tensor=inp, 
                       weights="/content/drive/MyDrive/Datasets/p2/DenseNet-BC-121-32-no-top.h5",
                       include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)

In [ ]:
pet_ids = train['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("/content/drive/MyDrive/Datasets/p2/train_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.to_csv('train_img_features.csv')
train_feats.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
86e1089a3,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
6296e909a,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
3422e4906,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
5842f1ff5,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
850a43f90,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685


In [ ]:
pet_ids = test['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("/content/drive/MyDrive/Datasets/p2/test_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.to_csv('test_img_features.csv')
test_feats.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
e2dfc2935,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
f153b465f,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
3c90f3f54,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
e02abc8a3,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
09f0df7d1,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,0.002864,0.025322,0.004963,0.000948,0.053315,0.074976,0.241215,0.002766,0.054716,0.078993,1.283727,0.003242,0.004274,0.002402,0.004979,0.136336,0.06618,0.045557,0.217914,0.153513,0.064352,0.024143,0.697426,0.021087,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685


In [ ]:
train_feats.to_csv('/content/drive/MyDrive/Datasets/p2/train_img_features2.csv')

In [ ]:
test_feats.to_csv('/content/drive/MyDrive/Datasets/p2/test_img_features2.csv')

In [ ]:
print(train_feats.shape, test_feats.shape)

(14993, 256) (3972, 256)


In [ ]:
train_feats.columns = ["img_feat{}".format(i) for i in range(256)]
test_feats.columns = ["img_feat{}".format(i) for i in range(256)]

train_feats["PetID"] = train_feats.index
test_feats["PetID"] = test_feats.index

train = pd.merge(train, train_feats, on="PetID")
test = pd.merge(test, test_feats, on="PetID")

print(train.shape, test.shape)

(14993, 280) (3972, 279)


In [ ]:
train.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,img_feat15,...,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,0.030244,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.00

In [ ]:
train.shape

(14993, 280)

In [ ]:
test.shape

(3972, 279)

In [ ]:
train.to_csv('/content/drive/MyDrive/Datasets/p2/train_with_image_features2.csv')

In [ ]:
test.to_csv('/content/drive/MyDrive/Datasets/p2/test_with_image_features2.csv')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/p2/train_with_image_features2.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/p2/test_with_image_features2.csv')

In [ ]:
train

,Unnamed: 0,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,...,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255
0,0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
1,1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
2,2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
3,3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.045237,0.051465,0.071165,0.063994,0.055685
4,4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0.001913,0.200123,0.034662,0.011279,0.22124,0.002751,0.001853,0.004002,0.114433,0.029524,0.092868,0.002139,0.020427,0.003696,0.015763,...,0.003215,0.0,0.165367,0.00083,0.17805,0.540773,0.246471,0.171875,0.068304,0.056133,0.033683,0.1418,0.034771,0.057069,0.013471,0.061857,0.018648,0.078102,0.015001,0.010246,0.011044,0.013416,0.000593,0.010083,0.00962,0.074501,0.02053,0.04781,0.311021,0.047721,0.017692,0.004894,0.045289,0.03007,0.013002,0.04